In [ ]:
import requests
import csv
import time

ACCESS_TOKEN = 'ACCESS TOKEN FROM GRAPH API'
USER_ID = 'USER ID FROM GRAPH API'

def get_instagram_posts(user_id, access_token, max_posts=10000):
    posts = []
    url = f"https://graph.facebook.com/v14.0/{user_id}/media?fields=id,caption,media_type,media_product_type&access_token={access_token}&limit=100"

    while url and len(posts) < max_posts:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            posts.extend(data.get('data', []))
            url = data.get('paging', {}).get('next')
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break
        time.sleep(0.3)

    return posts[:max_posts]

def get_insights(media_id, media_type, product_type, access_token):
    if product_type == "REELS":
        metrics = ["reach", "shares", "total_interactions","ig_reels_video_view_total_time"]
    elif media_type == "VIDEO":
        metrics = ["reach", "saved", "video_views"]
    elif media_type in ["IMAGE", "CAROUSEL_ALBUM"]:
        metrics = ["reach", "saved"]
    else:
        print(f"Skipping unsupported type: {media_type}/{product_type}")
        return {}

    metric_str = ",".join(metrics)
    url = f"https://graph.facebook.com/v14.0/{media_id}/insights?metric={metric_str}&access_token={access_token}"
    response = requests.get(url)
    if response.status_code == 200:
        return {item['name']: item['values'][0]['value'] for item in response.json().get('data', [])}
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return {}

def get_like_comment_counts(media_id, access_token):
    url = f"https://graph.facebook.com/v14.0/{media_id}?fields=like_count,comments_count&access_token={access_token}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            "like_count": data.get("like_count", 0),
            "comments_count": data.get("comments_count", 0)
        }
    else:
        print(f"Error fetching likes/comments: {response.status_code} - {response.text}")
        return {"like_count": 0, "comments_count": 0}

def export_to_csv(data, filename="instagram_insights.csv"):
    headers = [
        "Post ID", "Caption", "Media Type", "Product Type",
        "Reach", "Saved", "Video Views", "Shares", "Total Interactions",
        "Likes", "Comments"
    ]

    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)

    print(f"Exported insights to {filename}")

def main():
    posts = get_instagram_posts(USER_ID, ACCESS_TOKEN)
    if not posts:
        print("No posts found.")
        return

    rows = []

    for i, post in enumerate(posts, start=1):
        media_id = post.get('id')
        caption = post.get('caption', 'No caption')
        media_type = post.get('media_type', 'UNKNOWN')
        product_type = post.get('media_product_type', 'UNKNOWN')

        print(f"Processing post {i}/{len(posts)}: {media_id}")

        insights = get_insights(media_id, media_type, product_type, ACCESS_TOKEN)
        counts = get_like_comment_counts(media_id, ACCESS_TOKEN)

        row = [
            media_id,
            caption,
            media_type,
            product_type,
            insights.get("reach", 0),
            insights.get("saved", 0),
            insights.get("video_views", "-"),
            insights.get("shares", "-"),
            insights.get("total_interactions", "-"),
            counts["like_count"],
            counts["comments_count"]
        ]
        rows.append(row)

        time.sleep(0.2)

    export_to_csv(rows)

if __name__ == "__main__":
    main()


Processing post 1/117: 18056327990251660
Processing post 2/117: 18503436991034061
Processing post 3/117: 18033807797341012
Processing post 4/117: 18079831198792123
Processing post 5/117: 18102076816539274
Processing post 6/117: 18028395341461995
Processing post 7/117: 18067912678808922
Processing post 8/117: 18068497702768288
Processing post 9/117: 18082778002676119
Processing post 10/117: 17973773117838552
Processing post 11/117: 17895052743206296
Processing post 12/117: 18062512435860919
Processing post 13/117: 17949312968957932
Processing post 14/117: 18043872440158379
Processing post 15/117: 17945408900970940
Processing post 16/117: 18499115161012284
Processing post 17/117: 18065164438950081
Processing post 18/117: 18362187796126259
Processing post 19/117: 17899072776082722
Processing post 20/117: 18002369060749484
Processing post 21/117: 17957310119914224
Processing post 22/117: 18048272039466864
Processing post 23/117: 18037747304383475
Processing post 24/117: 18330693661091009
P

In [ ]:
import requests

ACCESS_TOKEN = 'EAAQqD5zUgMQBO4hU9cq5uKj2YAXxEHPYODLbZAYgs9JiHTuxmrF7GbECHxC2ZA86eP7Nhvfl6cTTNV717WdVmjrJOQFV0vkyxcd9ZA6JvZBu7CXCKZBegzUnKIOuDdEhSczeICbQ1g7ps4jPzZBIeOwINk1PPZAqblOFyIQJ0JCgFDZAlhEPF9PHA9ZA2bqTKJxM4'
USER_ID = '17841467453716484'

url = f"https://graph.facebook.com/v18.0/{USER_ID}/media?fields=media_type&access_token={ACCESS_TOKEN}"

image_count = 0
video_count = 0
carousel_count = 0
total_fetched = 0

while url:
    response = requests.get(url)
    data = response.json()

    if 'data' in data:
        for post in data['data']:
            total_fetched += 1
            media_type = post.get('media_type')

            if media_type == 'IMAGE':
                image_count += 1
            elif media_type == 'VIDEO':
                video_count += 1
            elif media_type == 'CAROUSEL_ALBUM':
                carousel_count += 1
    else:
        print("Error fetching media:", data)
        break

    url = data.get('paging', {}).get('next')

print(f"\nTotal posts fetched: {total_fetched}")
print(f"Image posts: {image_count}")
print(f"Video posts: {video_count}")
print(f"Carousel posts: {carousel_count}")


Total posts fetched: 117
Image posts: 3
Video posts: 110
Carousel posts: 4
